In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import matplotlib.pyplot as plt
%matplotlib inline


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import glob
import os
import numpy as np
import pickle
import re

labels = []
sentences = [[[] for _ in range(5)]]
count = 0
for file in glob.glob('./txt/*.txt', recursive=True):
    header =""
    print(file)
    with open(file, 'r') as f:
        sents = re.split(r'\.\s', f.read())
        for sentence in sents:
            if "Краткая информация" in sentence:
                header = 0
            elif "Диагностика" in sentence:
                header = 1
            elif "Лечение" in sentence:
                header = 2
            elif "Реабилитация" in sentence:
                header = 3
            elif "Профилактика" in sentence:
                header = 4
            if header != '':
                labels.append(header)
                sentences[count][header].append(sentence.replace('\n', ' '))
    sentences.append([[] for _ in range(5)])
    count += 1
       

./txt/КР472.txt
./txt/КР8.txt
./txt/КР116.txt
./txt/КР77.txt
./txt/КР76.txt
./txt/КР62.txt
./txt/КР329.txt
./txt/КР467.txt
./txt/КР249.txt
./txt/КР507.txt
./txt/КР277.txt
./txt/КР288.txt
./txt/КР317.txt
./txt/КР129.txt
./txt/КР48.txt
./txt/КР75.txt
./txt/КР289.txt
./txt/КР504.txt
./txt/КР266.txt
./txt/КР460.txt
./txt/КР138.txt
./txt/КР71.txt
./txt/КР58.txt
./txt/КР70.txt
./txt/КР111.txt
./txt/КР139.txt
./txt/КР475.txt
./txt/КР515.txt
./txt/КР501.txt
./txt/КР529.txt
./txt/КР339.txt
./txt/КР66.txt
./txt/КР73.txt
./txt/КР67.txt
./txt/КР98.txt
./txt/КР476.txt
./txt/КР338.txt
./txt/КР502.txt
./txt/КР565.txt
./txt/КР607.txt
./txt/КР161.txt
./txt/КР613.txt
./txt/КР149.txt
./txt/КР612.txt
./txt/КР160.txt
./txt/КР610.txt
./txt/КР16.txt
./txt/КР611.txt
./txt/КР588.txt
./txt/КР173.txt
./txt/КР601.txt
./txt/КР12.txt
./txt/КР614.txt
./txt/КР589.txt
./txt/КР574.txt
./txt/КР400.txt
./txt/КР372.txt
./txt/КР164.txt
./txt/КР602.txt
./txt/КР11.txt
./txt/КР39.txt
./txt/КР10.txt
./txt/КР603.txt
./txt/КР561

In [3]:
first_chapter = [sentences[i][0] for i in range(len(sentences))]
second_chapter =[sentences[i][1] for i in range(len(sentences))]
third_chapter =[sentences[i][2] for i in range(len(sentences))]
fourth_chapter =[sentences[i][3] for i in range(len(sentences))]
fifth_chapter =[sentences[i][4] for i in range(len(sentences))]

In [4]:
docs = first_chapter + second_chapter + third_chapter + fourth_chapter + fifth_chapter

In [5]:

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    text = re.sub("\d+", " ", text)
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token not in ['уровень','пациент','рекомендация']\
              and token != " " \
              and token.strip() not in punctuation]
    
    
    
    return tokens

In [6]:
processed = [[] for _ in range(len(docs))]
for doc in range(len(docs)):
    for sentence in docs[doc]:
        processed[doc]+=  preprocess_text(sentence)

In [7]:
processed[0]

['краткий',
 'информация',
 'определение',
 'перелом',
 'нижний',
 'челюсть',
 'повреждение',
 'нижний',
 'челюсть',
 'нарушение',
 'целостность',
 'этиология',
 'патогенез',
 'мирный',
 'время',
 'причина',
 'перелом',
 'нижний',
 'челюсть',
 'часто',
 'являться',
 'удар',
 'ушиб',
 'получать',
 'падение',
 'сдавление',
 'т',
 'д',
 'сравнительно',
 'редко',
 'встречаться',
 'огнестрельный',
 'перелом',
 'настоящий',
 'время',
 'возрастать',
 'частота',
 'транспортный',
 'бытовой',
 'травма',
 ' [ ,  ,  ]\n',
 'перелом',
 'нижний',
 'челюсть',
 'обычно',
 'возникать',
 'результат',
 'воздействие',
 'сила',
 'который',
 'превышать',
 'физический',
 'возможность',
 'костный',
 'ткань',
 'перелом',
 'называть',
 'травматический',
 ' [ ]\n',
 'выделять',
 'четыре',
 'механизм',
 'перелом',
 'нижний',
 'челюсть',
 'перегиб',
 'сдвиг',
 'сжатие',
 'отрыв',
 'нижний',
 'челюсть',
 'иметь',
 'форма',
 'дуга',
 'область',
 'угол',
 'подбородочный',
 'отверстие',
 'клык',
 'область',
 'основани

In [8]:
id2word = corpora.Dictionary([doc for doc in processed])

# Create Corpus
corpus = []
for doc in processed:
    corpus.append(id2word.doc2bow(doc))


In [9]:
len(corpus)
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 2),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 4),
 (17, 4),
 (18, 7),
 (19, 3),
 (20, 1),
 (21, 3),
 (22, 1),
 (23, 1),
 (24, 5),
 (25, 3),
 (26, 1),
 (27, 2),
 (28, 6),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 1),
 (33, 1),
 (34, 4),
 (35, 4),
 (36, 4),
 (37, 1),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 2),
 (42, 2),
 (43, 2),
 (44, 1),
 (45, 5),
 (46, 1),
 (47, 2),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 2),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 2),
 (58, 1),
 (59, 5),
 (60, 1),
 (61, 1),
 (62, 5),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 2),
 (68, 1),
 (69, 3),
 (70, 1),
 (71, 4),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 2),
 (83, 1),
 (84, 2),
 (85, 5),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 2),
 (90, 1),
 (91, 2),
 (92, 7),
 (93, 1),
 (94, 1),
 (95, 2),
 (96, 1),
 (97, 1),
 (98, 2),
 (99, 3),
 (100, 2),

In [10]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, passes=20)


In [11]:
pprint(lda.print_topics())

[(0,
  '0.021*"терапия" + 0.012*"лечение" + 0.011*"доказательство" + '
  '0.011*"убедительность" + 0.010*"достоверность" + 0.010*"мг" + '
  '0.009*"рекомендоваться" + 0.008*"год" + 0.006*"проведение" + 0.006*"**  "'),
 (1,
  '0.030*"медицинский" + 0.020*"помощь" + 0.016*"онкологический" + '
  '0.016*"врач" + 0.013*"лечение" + 0.011*"организация" + 0.011*"заболевание" '
  '+ 0.009*"убедительность" + 0.009*"доказательство" + 0.009*"достоверность"'),
 (2,
  '0.011*"удда" + 0.011*"уур" + 0.010*"рекомендоваться" + 0.008*"риск" + '
  '0.008*"еок" + 0.007*"терапия" + 0.006*"рекомендовать" + 0.005*"комментарий" '
  '+ 0.005*"лечение" + 0.004*"доза"'),
 (3,
  '0.010*"убедительность" + 0.010*"доказательство" + 0.010*"достоверность" + '
  '0.009*"рекомендоваться" + 0.008*"комментарий" + 0.007*"лечение" + '
  '0.007*"перелом" + 0.005*"легочный" + 0.005*"беременность" + '
  '0.005*"операция"'),
 (4,
  '0.013*"рекомендоваться" + 0.013*"доказательство" + 0.013*"достоверность" + '
  '0.012*"сд" + 0.01

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

gensimvis.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.009672 -0.072503       1        1  17.020975
7      0.110884 -0.046258       2        1  14.537845
9      0.230759  0.071389       3        1  12.988925
1     -0.004809 -0.171379       4        1  11.415185
2     -0.077509  0.119885       5        1   9.534348
4     -0.006365  0.065911       6        1   8.333290
0     -0.055365 -0.031293       7        1   8.246170
3     -0.052283  0.058570       8        1   7.941649
8     -0.076489  0.009838       9        1   6.622699
5     -0.078496 -0.004160      10        1   3.358914, topic_info=                Term         Freq        Total Category  logprob  loglift
1125     медицинский  3740.000000  3740.000000  Default  30.0000  30.0000
3118  онкологический  2025.000000  2025.000000  Default  29.0000  29.0000
1302         терапия  7194.000000  7194.000000  Default  28.0000  28.0000
392           помощь  2863.000000  2863.000000  Default  27.0000  27.0000
617          опухоль  5480.000000  5480.000000  Default  26.0000  26.0000
...              ...          ...          ...      ...      ...      ...
169          ребенок   129.874647  1549.251466  Topic10  -5.5518   0.9146
3526      проведение   136.774489  3889.307777  Topic10  -5.5000   0.0459
845            орган   122.967972  1536.100363  Topic10  -5.6064   0.8685
2012      активность   117.988414   813.944066  Topic10  -5.6478   1.4622
218            форма   113.464064  1320.914238  Topic10  -5.6869   0.9390

[868 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
17120      5  0.971751    ) [ , ]\n
5578       1  0.021620      ) [ ]\n
5578       3  0.032431      ) [ ]\n
5578       5  0.929677      ) [ ]\n
8454       1  0.087473          ‒  
...      ...       ...          ...
236        9  0.065429     являться
236       10  0.029008     являться
11454      2  0.016427           як
11454      3  0.008213           як
11454      9  0.969186           як

[3099 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 10, 2, 3, 5, 1, 4, 9, 6])

In [13]:
def create_corpus(text):
    processed = [[] for _ in range(len(text))]
    for doc in range(len(text)):
        for sentence in text[doc]:
            processed[doc]+=  preprocess_text(sentence)
    id2word = corpora.Dictionary([doc for doc in processed])

    corpus = []
    for doc in processed:
        corpus.append(id2word.doc2bow(doc))
    return corpus

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
corpus1 = create_corpus(first_chapter)
corpus2 = create_corpus(second_chapter)
corpus3 = create_corpus(third_chapter)
corpus4 = create_corpus(fourth_chapter)
corpus5 = create_corpus(fifth_chapter)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
def count_probs(corp):
    probs  = [[] for _ in range(10)]
    for doc in corp:
        for prob in lda[doc]:
            probs[prob[0]].append(prob[1]) 
    return [sum(prob)/len(first_chapter) for prob in probs]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
probs1 = count_probs(corpus1)
probs2 = count_probs(corpus2)
probs3 = count_probs(corpus3)
probs4 = count_probs(corpus4)
probs5 = count_probs(corpus5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
probs1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.02941053166720994,
 0.02422174438283112,
 0.04577584498556408,
 0.10103395606043952,
 0.04810551339044021,
 0.028462729951333716,
 0.028061293633158742,
 0.049026141560377455,
 0.02467398939532994,
 0.6196740396188561]

In [18]:
probs2

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.008899378485580874,
 0.06513638847876758,
 0.05702354043126988,
 0.12198645782453069,
 0.047061168577676345,
 0.021046829911378715,
 0.05572818144163966,
 0.04529191983984949,
 0.011087607283740353,
 0.5625616809320167]

In [19]:
probs3

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.011186704243472694,
 0.03879995181186665,
 0.08495002286784219,
 0.14373436102853018,
 0.05172346318526381,
 0.0292238901540108,
 0.06624730863779255,
 0.06691208639397071,
 0.011272062481375487,
 0.4922123422636788]

In [20]:
probs4

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.042528772974640305,
 0.06472547234934287,
 0.034863727982844823,
 0.14558023900432876,
 0.05445287978419891,
 0.015067339659883426,
 0.02851414928466258,
 0.01738831856613152,
 0.01613890138846001,
 0.5712140113357785]

In [21]:
probs5

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.008320207586346646,
 0.057566979833873065,
 0.05124226954253115,
 0.11381992650323013,
 0.042254434501719194,
 0.02779223319672092,
 0.07335536577049974,
 0.02210582568470014,
 0.010514719973830782,
 0.5899053609946511]